In [1]:
from dev_interaction_util import *



In [2]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Behavioral data


In [3]:
dropbox_data_dir = config['dropbox_data_dir']
analysis_data, outcome_measures = load_and_preprocess_data(dropbox_data_dir)
#analysis_data_imputed = impute_data(analysis_data)

EmptyDataError: No columns to parse from file

## Neural data
TBD!



## Outcome data

In [ ]:
synthetic_data, group_assignments = generate_synthetic_dev_outcomes_and_interactions(outcome_measures, analysis_data)

interaction_effect_df = synthetic_data['X_weights']
outcome_measures = synthetic_data['y']


['ni' 'san']
[1.28335298 0.42953651]
['san' 'san' 'ni' 'ichi' 'san' 'san' 'ichi' 'san' 'san' 'san' 'ni' 'ichi'
 'ichi' 'ichi' 'ichi' 'san' 'san' 'san' 'ichi' 'ichi' 'san' 'san' 'ni'
 'ni' 'ni' 'ni' 'ni' 'ni' 'ni' 'san' 'ni' 'san' 'ni' 'ichi' 'ni' 'san'
 'ni' 'ichi' 'san' 'ni' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ni' 'ni'
 'san' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ichi' 'ni' 'ni' 'ni' 'ichi' 'san'
 'ni' 'ni' 'ichi' 'ni' 'ichi' 'san' 'ni' 'ni' 'ni' 'san' 'ichi' 'ni' 'san'
 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'ichi' 'san' 'ichi' 'san' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'san' 'san' 'san' 'ichi'
 'ni' 'san' 'ichi' 'ichi' 'san' 'ni' 'ichi' 'san' 'ni' 'ni' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'ichi' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ichi'
 'ni' 'ni' 'ichi' 'ni' 'ni' 'ichi' 'ichi' 'san' 'san' 'ni' 'ichi' 'ni'
 'ichi' 'ichi' 'san' 'ichi' 'ni' 'san' 'san' 'ni' 'ni' 'san' 'san' 'san'
 'ichi' 'san' 'ni' 'san' 'ichi' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ni' 'ni'
 'ichi' 'ni' 'ichi' '

## Tidy data

In [57]:
outcome_nas,outcome_measures_nona, predictor_data_nona, group_assignment_onehots_nonan, group_assignments_nona = get_outcome_changes(outcome_measures, analysis_data, group_assignments)


(275, 76)
(275, 76)


In [63]:
outcome_measures_nona.columns

Index(['cancer_promoting_minus_preventing_FFQ_w1', 'bf_1',
       'FFQ_v2_Mean_Energy_w1', 'bf_2',
       'cancer_promoting_minus_preventing_FFQ_w2', 'FFQ_v2_Mean_Energy_w2',
       'd_bf', 'd_cancer_promoting_minus_preventing_FFQ',
       'd_FFQ_v2_Mean_Energy'],
      dtype='object')

outcome_measure
d_bf                                       0.091314
d_cancer_promoting_minus_preventing_FFQ    0.303603
Name: correlation, dtype: float64

OK so. probably that correlation has something to do with it!

In [75]:
#get the correlations between outcome measures and the predictors; do this from scratch using a suitable correlation function
#do this pairwise, ignoring na values in each paring
outcome_measure_correlations = pd.DataFrame(columns=['outcome_measure','predictor','correlation'])
for predictor in predictor_data_nona.columns:
    for outcome_measure in ['d_bf', 'd_cancer_promoting_minus_preventing_FFQ']:
        measured_corr = outcome_measures_nona[outcome_measure].corr(predictor_data_nona[predictor],method='pearson')
        #print("Correlation between " + outcome_measure + " and " + predictor + " is " + str(measured_corr))
        measured_corr_df = {'outcome_measure':outcome_measure,'predictor':predictor,'correlation':measured_corr}
        #append the df using concat
        outcome_measure_correlations = pd.concat([outcome_measure_correlations,pd.DataFrame([measured_corr_df])])
        


In [77]:
#group correlations by outcome_measure and get the mean of the absolutes of all of them
outcome_measure_correlations.groupby('outcome_measure').correlation.apply(lambda x: np.mean(np.abs(x)))

outcome_measure
d_bf                                       0.091314
d_cancer_promoting_minus_preventing_FFQ    0.303603
Name: correlation, dtype: float64

# Predict change

In [26]:
#Need to add imputation in the dataset below!


def score_and_present(predictor_data,
                      outcome_measure,
                      group_assignments
                      ):
    scoring_data = do_scoring_loop(X=predictor_data, y= outcome_measure, 
                groups = group_assignments_nona, 
                hyperparameter_selection_on_fold=do_hyperparameter_selection_loop_fast,
                outer_folds=5)

    scores = scoring_data['scores']
    best_models = scoring_data['best_models']
    best_params_df_list = scoring_data['best_params_df_list']
    raw_cv_results_list = scoring_data['raw_cv_results_list']

    print("scores:")
    print(scores)
    overall_score = np.mean(scores)
    print("overall_score:")
    print(overall_score)


    
    best_model = get_best_model(summarize_overall_df_results(raw_cv_results_list))
    final_fit = do_final_fit(X=predictor_data, y= outcome_measure, final_model=best_model, impute_missing=True)

    pd_imputed, X_was_imputed = apply_imputer(predictor_data)

    final_results = present_model_results(X=pd_imputed, final_fit=final_fit, y=outcome_measure)

    #print rows of final_results where feature_name is the list of features to check
    base_regressors = interaction_effect_df.predictor[interaction_effect_df.interaction_effect!=0]
    regressors_to_check = [x+y for y in ['','*ni','*san'] for x in base_regressors]
    final_results['planned_regression'] = final_results['predictor'].isin(regressors_to_check)

    #this is only for synethetic analysis
    #present_results_vs_ground_truth_cors(pd_imputed,outcome_measure,group_assignments,final_results,base_regressors)

    return({
        'final_results':final_results,
        'final_fit':final_fit,
        'overall_score':overall_score
        })

## Next: automate this flow into the different analyses we need to run, with different predictors and different DVs


In [27]:
condition_cols = ['ni','san']
inddiff_cols = [
        'ACES_sum',
'BFI_agreeableness',
'BFI_conscientiousness',
'BFI_extraversion',
'BFI_neuroticism',
'BFI_openness',
'IMI_effort_importance',
'IMI_interest_enjoyment',
'NCS_total',
'PLAN_cognitive_strategies',
'PLAN_mental_forecasting',
'PLAN_temporal_orientation',
'RMQ_assessment',
'TESQ_E_sum',
'SRHI_healthy_minus_unhealthy',
'RTFS_f1_minus_f2',
'cancer_promoting_minus_preventing_craved_FCI',
'cancer_promoting_minus_preventing_liked_FCI',
'cSES',
'age365',
'birthsex_factor_Male',
'education_own',
'SST_SSD',
'SST_mean_ssrt_0',
'ROC_Crave_Regulate_Minus_Look',
'WTP_unhealthy_minus_healthy'
    ]
interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_only': condition_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols
}


outcome_vars_to_try = [ 'd_bf', 'd_cancer_promoting_minus_preventing_FFQ'
                       #,'d_ASA24'
                       ]

In [31]:
final_results = score_and_present(
            predictor_data_nona[condition_cols],
            outcome_measures_nona['d_bf'],
            group_assignments_nona
                  )

outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [2], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [2], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
outer split1
Pi

doing permutation test on importance; this may take time.
Number of selected features: 2


,predictor,coef,feature_importance,fa_abs
0,ni,0.897607,0.087663,0.087663
1,san,0.307429,0.006227,0.006227


In [33]:
final_results['overall_score']

8.075287760822558e-05

In [45]:

def do_full_analysis(outcome_var):
    model_outcomes = {}
    for psk in predictor_sets.keys():
        predictor_set = predictor_sets[psk]
        print(" attempting to predict " + outcome_var + " with " + str(len(predictor_set)) + " predictors in the set " + psk)
        print('predictors in that set are ' + " ".join(predictor_set))
        model_outcomes[psk] = score_and_present(
            predictor_data_nona[predictor_set],
            outcome_measures_nona[outcome_var],
            group_assignments_nona
                  )
        
    #get a dictionary of the overall scores for each model
    model_outcomes_comparison = { k:
        model_outcomes[k]['overall_score'] for k in model_outcomes.keys()}

    print(model_outcomes_comparison)
        
    return(model_outcomes)

    
    


## Body Fat Percentage

In [46]:
outcome_vars_to_try

['d_bf', 'd_cancer_promoting_minus_preventing_FFQ']

In [47]:
model_outcomes = do_full_analysis('d_bf')

 attempting to predict d_bf with 2 predictors in the set condition_only
predictors in that set are ni san
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [2], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [2], '

doing permutation test on importance; this may take time.
Number of selected features: 2


,predictor,coef,feature_importance,fa_abs
0,ni,0.897607,0.07771,0.07771
1,san,0.307429,0.01290,0.01290


 attempting to predict d_bf with 28 predictors in the set condition_inddiff
predictors in that set are ni san ACES_sum BFI_agreeableness BFI_conscientiousness BFI_extraversion BFI_neuroticism BFI_openness IMI_effort_importance IMI_interest_enjoyment NCS_total PLAN_cognitive_strategies PLAN_mental_forecasting PLAN_temporal_orientation RMQ_assessment TESQ_E_sum SRHI_healthy_minus_unhealthy RTFS_f1_minus_f2 cancer_promoting_minus_preventing_craved_FCI cancer_promoting_minus_preventing_liked_FCI cSES age365 birthsex_factor_Male education_own SST_SSD SST_mean_ssrt_0 ROC_Crave_Regulate_Minus_Look WTP_unhealthy_minus_healthy
outer split0


/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
scores:
[0

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.
Number of selected features: 16


,predictor,coef,feature_importance,fa_abs
29,ACES_sum*san,-0.727862,0.067076,0.067076
35,BFI_extraversion*san,0.719851,0.063594,0.063594
48,PLAN_mental_forecasting*ni,0.684014,0.054243,0.054243
17,RTFS_f1_minus_f2,-0.527450,0.038589,0.038589
10,NCS_total,-0.471688,0.031311,0.031311
13,PLAN_temporal_orientation,0.278116,0.018155,0.018155
2,ACES_sum,-0.260162,0.009643,0.009643
63,cancer_promoting_minus_preventing_liked_FCI*san,-0.216391,0.008606,0.008606
78,WTP_unhealthy_minus_healthy*ni,-0.191974,0.008370,0.008370
55,TESQ_E_sum*san,0.215864,0.007778,0.007778


{'condition_only': 8.075287760822558e-05, 'condition_inddiff': -0.010013318224489986, 'condition_inddiff_interactions': -0.009234467392482525}


## FFQ

In [52]:
model_outcomes = do_full_analysis('d_cancer_promoting_minus_preventing_FFQ')

 attempting to predict d_cancer_promoting_minus_preventing_FFQ with 2 predictors in the set condition_only
predictors in that set are ni san
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [2], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e

doing permutation test on importance; this may take time.
Number of selected features: 2


,predictor,coef,feature_importance,fa_abs
0,ni,0.605600,1.889327,1.889327
1,san,0.189365,0.194790,0.194790


 attempting to predict d_cancer_promoting_minus_preventing_FFQ with 28 predictors in the set condition_inddiff
predictors in that set are ni san ACES_sum BFI_agreeableness BFI_conscientiousness BFI_extraversion BFI_neuroticism BFI_openness IMI_effort_importance IMI_interest_enjoyment NCS_total PLAN_cognitive_strategies PLAN_mental_forecasting PLAN_temporal_orientation RMQ_assessment TESQ_E_sum SRHI_healthy_minus_unhealthy RTFS_f1_minus_f2 cancer_promoting_minus_preventing_craved_FCI cancer_promoting_minus_preventing_liked_FCI cSES age365 birthsex_factor_Male education_own SST_SSD SST_mean_ssrt_0 ROC_Crave_Regulate_Minus_Look WTP_unhealthy_minus_healthy
outer split0


/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
outer spli

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17e7ed4e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
scores:
[0

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


KeyboardInterrupt: 